In [1]:
import tensorflow as tf

Layers:

In [2]:
class CustomLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(CustomLayer, self).__init__( **kwargs)
        """
        設定參數的地方
        """
        
    def build(self, input_shape):
        """
        建立權重的地方(透過add_weight 方法)
        參數:
            input_shape: 輸入大小
        """
        
    def call(self, inputs):
        """
        定義網路前向傳遞(運算)的地方。
        參數:
            inputs: 輸入網路的資料
        """
    def get_config(self):
        """
        (選擇)如果你要支援序列化，要在這裡定義，它會傳回層的構建參數
        """

Loss:

In [3]:
def custom_loss(y_true, y_pred):
    """
    定義loss 計算在這個地方
    參數:
        y_true(真實值): 傳入這筆資料的答案
        y_pred(預測值): 傳入這筆資料網路預測的結果
    """
    return loss

Metrics:

In [5]:
class CustomMetrics(tf.keras.metrics.Metric):
    def __init__(self, name='custom_metrics', **kwargs):
        super(CustomMetrics, self).__init__(name=name, **kwargs)
        """
        所有指標函數使用到的狀態變數都需在這裡建立
        參數:
            name: 指標函數的名稱
        """
        
    def update_state(self, y_true, y_pred, sample_weight=None):
        """
        使用y_true(真實值) 與 y_pred(預測值) 來計算更新狀態函數
        參數:
            y_true(真實值): 傳入這筆資料的答案
            y_pred(預測值): 傳入這筆資料網路預測的結果
            sample_weight: 對樣本的權重，通常用在序列模型
        """
    def result(self):
        """
        使用狀態變數計算最終的結果
        """
    def reset_states(self):
        """
        重新初始化指標函數(狀態函數)
        """

Callbacks

class CustomCallbacks(tf.keras.callbacks.Callback):
    def on_epoch_(begin|end)(self, epoch, logs=None):
        """
        每一個epoch 開始或結束，執行這段程式
        參數:
            epoch: 目前的epoch
            logs: 傳入dict 格式的記錄資訊，例如: loss, val_loss 等
        """
    def on_(train|test|predict)_begin(self, logs=None):
        """
        fit, evaluate, predict 任務開始時，執行這段程式
        參數:
            logs: 傳入dict 格式的記錄資訊，例如: loss, val_loss 等
        """
    def on_(train|test|predict)_end(self, logs=None):
        """
        fit, evaluate, predict 任務結束時，執行這段程式
        參數:
            logs: 傳入dict 格式的記錄資訊，例如: loss, val_loss 等
        """
    def on_(train|test|predict)_batch_begin(self, batch, logs=None):
        """
        fit, evaluate, predict 任務的每一個batch 開始前執行這段程式
        參數:
            batch: 目前的batch
            logs: 傳入dict 格式的記錄資訊，例如: loss, val_loss 等
        """
    def on_(train|test|predict)_batch_end(self, batch, logs=None):
        """
        fit, evaluate, predict 任務的每一個batch 結束後執行這段程式
        參數:
            batch: 目前的batch
            logs: 傳入dict 格式的記錄資訊，例如: loss, val_loss 等
        """

Making Conv layer with keras API vs customize API

In [7]:
tf.keras.layers.Conv2D(64, 3, activation='relu',
                       kernel_initializer='glorot_uniform')

In [8]:
class CustomConv2D(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, strides=(1, 1), padding="VALID",
                 **kwargs):
        super(CustomConv2D, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.strides = (1, *strides, 1)
        self.padding = padding
        
    def build(self, input_shape):
        kernel_h , kernel_w = self.kernel_size
        input_dim = input_shape[-1]
        #建立卷積層的權重值(weights)
        self.w = self.add_weight(name='kernel',
                                 shape=(kernel_h, kernel_w, input_dim, self.filters),
                                 initializer='glorot_uniform', #設定初始化方法
                                 trainable=True) #設定這個權重是否能訓練(更動)
        #建立卷積層的偏差值(bias)
        self.b = self.add_weight(name='bias',
                                 shape=(self.filters,),
                                 initializer='zeros', #設定初始化方法
                                 trainable=True) #設定這個權重是否能訓練(更動)
    
    def call(self, inputs):
        #卷積運算
        x = tf.nn.conv2d(inputs, self.w, self.strides, padding=self.padding)
        x = tf.nn.bias_add(x, self.b) #加上偏差值
        x = tf.nn.relu(x)
        return x

Loss Categorical Cross Entropy

In [9]:
tf.keras.losses.CategoricalCrossentropy()

In [10]:
def custom_categorical_crossentropy(y_true, y_pred):
    x = tf.reduce_mean(-tf.reduce_sum(y_true * tf.log(y_pred),
                        reduction_indices=[1]))
    return x

實際不使用上面的function, 下面的經過優化，會更快和穩定

In [12]:
def custom_categorical_crossentropy(y_true, y_pred):
    x = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    return x

Metrics Categorical_accuracy=Correct_numbers/Total_numbers

In [13]:
tf.keras.metrics.CategoricalAccuracy()

In [14]:
class CustomCategoricalAccuracy(tf.keras.metrics.Metric):
    def __init__(self, name='custom_categorical_accuracy', **kwargs):
        super(CustomCategoricalAccuracy, self).__init__(name=name, **kwargs)
        self.correct = self.add_weight('correct_numbers', initializer='zeros')
        self.total = self.add_weight('total_numbers', initializer='zeros')
        
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.argmax(y_true,axis=-1)
        y_pred = tf.argmax(y_pred,axis=-1)
        values = tf.equal(y_true, y_pred)
        values = tf.cast(values, tf.float32)
        values_sum = tf.reduce_sum(values)
        num_values = tf.cast(tf.size(values), tf.float32)
        self.correct.assign_add(values_sum)
        self.total.assign_add(num_values)
        
    def result(self):
        return tf.math.divide_no_nan(self.correct, self.total)
    
    def reset_states(self):
        self.correct.assign(0.)
        self.total.assign(0.)

Callback

In [15]:
tf.keras.callbacks.ModelCheckpoint('logs/models/save.h5')

In [16]:
class SaveModel(tf.keras.callbacks.Callback):
    def __init__(self, weights_file, monitor='loss', mode='min',
                 save_weights_only=False):
        super(SaveModel, self).__init__()
        #設定模型權重儲存路徑
        self.weights_file = weights_file
        #設定要監察的數值
        self.monitor = monitor
        #監察模式: loss 用 min, Accuracy 用 max
        self.mode = mode
        #只儲存網路權重或儲存整個網路模型(包括 layer, compile等)
        self.save_weights_only = save_weights_only
        if mode == 'min':
            #設定best 為無限大
            self.best = np.Inf
        else:
            #設定best 為負無限大
            self.best = -np.Inf
        
    #儲存網路模型函數
    def save_model(self):
        if self.save_weights_only:
            #只儲存網路權重
            self.model.save_weights(self.weights_file)
        else:
            #儲存整個網路模型(包括 layer, compile等)
            self.model.save(self.weights_file)
    
    def on_epoch_end(self, epoch, logs=None):
        #從logs中讀取監測值
        monitor_value = logs.get(self.monitor)
        #如果監察值有降低或升高(取決於你的mode設定)，則儲存網路模型
        if self.mode == 'min' and monitor_value < self.best:
            self.save_model()
            self.best = monitor_value
        elif self.mode == 'max' and monitor_value > self.best:
            self.save_model()
            self.best = monitor_value